Fitting catalogues of data with Bagpipes
================================

A situation which commonly arises is having a catalogue of objects with the same observations which we wish to fit with the same model. You could wrap the fitting commands from the previous three examples in a for loop in order to do this yourself, but Bagpipes provides a catalogue fitting interface which makes this easier, and allows for parallelisation, in the sense that different objects will automatically be parcelled out to processes running on different cores. 

Setting up
------------

We'll use the setup from Example 3 to demonstrate how catalogue fitting works, first of all let's load the data and generate the fit instructions dictionary.

In [1]:
import numpy as np 
import bagpipes as pipes

from astropy.io import fits
from glob import glob

def load_uvista(ID):
    """ Load UltraVISTA photometry from catalogue. """

    # load up the relevant columns from the catalogue.
    hdulist = np.loadtxt("UltraVISTA_catalogue.cat",
                         usecols=(0,3,4,5,6,7,8,9,10,11,12,13,14,15,
                                  16,17,18,19,20,21,22,23,24,25,26))

    # Check the object is in the catalogue.
    if np.min(np.abs(hdulist[:,0] - int(ID))) != 0:
        sys.exit("Object not found in catalogue")

    tablerow = np.argmin(np.abs(hdulist[:,0] - int(ID)))

    # Extract the object we're interested in from the catalogue.
    phot_fluxes = hdulist[tablerow, 1:13]
    phot_fluxerrs = hdulist[tablerow, 13:25]

    phot = np.zeros(len(phot_fluxes)*2)
    phot.shape = (len(phot_fluxes), 2)
    
    # Convert to microjanskys
    phot[:,0] = phot_fluxes*10**29
    phot[:,1] = phot_fluxerrs*10**29

    # blow up the errors associated with any N/A points in the phot
    for i in range(len(phot)):
        if ((phot[i, 0] == 0. or phot[i, 1] <= 0) 
                or (phot[i, 1] > 0 and -phot[i, 0] >= 2*phot[i, 1])):
            phot[i,:] = [0., 9.9*10**99.]

    return phot


uvista_filt_list = ["uvista/CFHT_u.txt",
                    "uvista/CFHT_g.txt",
                    "uvista/CFHT_r.txt",
                    "uvista/CFHT_i+i2.txt",
                    "uvista/CFHT_z.txt",
                    "uvista/subaru_z",
                    "uvista/VISTA_Y.txt",
                    "uvista/VISTA_J.txt",
                    "uvista/VISTA_H.txt",
                    "uvista/VISTA_Ks.txt",
                    "uvista/IRAC1",
                    "uvista/IRAC2"]


exp = {}                          # Tau-model star-formation history component
exp["age"] = (0.1, 15.)           # Vary age between 100 Myr and 15 Gyr, in practice 
                                  # the code automatically limits this to the age of
                                  # the Universe at the observed redshift.

exp["tau"] = (0.3, 10.)           # Vary tau between 300 Myr and 10 Gyr
exp["massformed"] = (1., 15.)     # vary log_10(M*/M_solar) between 1 and 15
exp["metallicity"] = (0., 2.5)    # vary Z between 0 and 2.5 Z_oldsolar

dust = {}                         # Dust component
dust["type"] = "Calzetti"         # Define the shape of the attenuation curve
dust["Av"] = (0., 2.)             # Vary Av between 0 and 2 magnitudes

fit_info = {}                     # The fit instructions dictionary
fit_info["redshift"] = (0., 10.)  # Vary observed redshift from 0 to 10
fit_info["exponential"] = exp   
fit_info["dust"] = dust

Basic catalogue fitting
--------------------------

In the most basic case all you need is a list of IDs. You can pass this and a few other arguments to the catalogue_fit object and it takes care of the rest.

In [2]:
IDs = np.loadtxt("UltraVISTA_catalogue.cat", usecols=0).astype(int).astype(str)

cat_fit = pipes.catalogue_fit(IDs, fit_info, load_uvista, spectrum_exists=False,
                              cat_filt_list=uvista_filt_list, run="cat_fit")

That's all there is to it, just call the fit method in the same way as you would for the regular fit class and it loops through the catalogue fitting each object.

In [3]:
cat_fit.fit(sampler="pmn")


Bagpipes: fitting object 90052 with PyMultiNest

  analysing data from pipes/posterior/cat_fit/90052-.txt

Bagpipes: fitting complete in 119.3 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        1.284      1.637      1.897
exponential:age                1.621      2.034      3.422
exponential:massformed        10.677     10.768     10.908
exponential:metallicity        0.348      0.956      2.004
exponential:tau                0.328      0.421      0.878
redshift                       0.746      0.795      0.852


Bagpipes: 1 out of 10 objects completed.


/Users/adam/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)



Bagpipes: fitting object 96976 with PyMultiNest

  analysing data from pipes/posterior/cat_fit/96976-.txt

Bagpipes: fitting complete in 157.1 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.091      0.249      0.442
exponential:age                2.418      3.508      5.205
exponential:massformed        10.813     10.904     11.027
exponential:metallicity        0.224      0.416      0.701
exponential:tau                0.394      0.596      0.915
redshift                       0.846      0.896      0.954


Bagpipes: 2 out of 10 objects completed.

Bagpipes: fitting object 102216 with PyMultiNest

Bagpipes: lnlike was nan, replaced with zero probability.
  analysing data from pipes/posterior/cat_fit/102216-.txt

Bagpipes: fitting complete in 146.7 seconds.

Parameter                          Posterior percentiles
   

The real advantage here is that if you set another instance of this file running in a different terminal window, bagpipes will automatically share the objects in the catalogue out between these two (or more) processes. Processes can be started and stopped at any time and everything should carry on working. The only exception is starting more than one process at exactly the same time which can lead to conflicts. If you're setting a large number of parallel processes going at once, try adding a random time delay to the beginning of the file to avoid this.

A summary output catalogue will be generated in the pipes/cats directory. The code generates a .lock file in pipes/cats/run which prevents the code fitting the same object again. If you want to re-fit an object (e.g. the process that was fitting it crashed) you'll need to delete the .lock file and any files under pipes/posterior related to the object.
    
If a bunch of processes crashed and things have got messed up you can either do "rm -r pipes/cats/run\*" and "rm -r pipes/posterior/run\*" if you don't care about objects that already finished, or if you do, kill running processes and use the pipes.clean_cat(run) command. This removes progress on objects which aren't finished and automatically deletes their .lock files, which should allow you to resume from the last point things were working.

More complex options
--------------------------

There are a few other options that might come in handy. For example, if you have a list of spectroscopic redshifts for the objects you're fitting you might wish to fix the redshift of each fit to a different value. You can do this using the cat_redshifts and fix_redshifts keyword arguments.

In [4]:
redshifts = np.loadtxt("UltraVISTA_catalogue.cat", usecols=-2)

cat_fit = pipes.catalogue_fit(IDs, fit_info, load_uvista, spectrum_exists=False,
                              cat_filt_list=uvista_filt_list, run="cat_fit",
                              cat_redshifts=redshifts, fix_redshifts=True)

It is important to note that the redshifts will not be fixed automatically just by specifying the cat_redshifts keyword argument. The fix_redshifts keyword argument being True is what does this.

If instead you want to vary the redshift within a small range around the spectroscopic redshift you can pass a float to fix_redshifts. E.g. if you want the redshift to be fitted within a range of 0.05 either side of the value in cat_redshifts you can do the following.

In [5]:
cat_fit = pipes.catalogue_fit(IDs, fit_info, load_uvista, spectrum_exists=False,
                              cat_filt_list=uvista_filt_list, run="cat_fit",
                              cat_redshifts=redshifts, fix_redshifts=0.05)

Finally, if you have a bunch of different objects with different photometry that you want to fit with the same model you can pass a list of filter lists to catalogue_fit as the cat_filt_list keyword argument. If you do this you need to set the vary_filt_list keyword argument to True, and the code will expect the first entry in cat_filt_list to be the filter list for the first object and so on. We can set this up using the same filter list for each object just to demonstrate:

In [6]:
list_of_filt_lists = [uvista_filt_list] * 10

cat_fit = pipes.catalogue_fit(IDs, fit_info, load_uvista, spectrum_exists=False,
                              cat_filt_list=list_of_filt_lists, run="cat_fit",
                              cat_redshifts=redshifts, fix_redshifts=0.05,
                              vary_filt_list=True)